#***Student Name: Ji Qi*** , ***Session B1***




# Calculating Risk Exposures at the Start of the Year 2022 for 100 Stocks

### Import libraries and packages

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Upload WRDS CRSP Stock Return Data
* CSV file contains monthly stock returns for 100 stocks

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
stockret = pd.read_csv('/content/drive/MyDrive/BA_870/HW/3/100-Stocks-Returns.csv')

#### Examine varaibles in dataframe

In [68]:
stockret.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PERMNO  6000 non-null   int64  
 1   date    6000 non-null   int64  
 2   TICKER  6000 non-null   object 
 3   RET     6000 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 187.6+ KB


#### Print out header of dataframe
* Note that BWXT data appears first (60 monthly observations)

In [69]:
stockret.head()

,PERMNO,date,TICKER,RET
0,10220,20170131,BWXT,0.045088
1,10220,20170228,BWXT,0.119306
2,10220,20170331,BWXT,0.026916
3,10220,20170428,BWXT,0.032983
4,10220,20170531,BWXT,-0.009355


#### Print out "tail" (the last 5 obs) of dataframe
* Note that BLDR data is at the end (60 monthly observations)

In [70]:
stockret.tail()

,PERMNO,date,TICKER,RET
5995,90720,20210831,BLDR,0.197528
5996,90720,20210930,BLDR,-0.029086
5997,90720,20211029,BLDR,0.126208
5998,90720,20211130,BLDR,0.191694
5999,90720,20211231,BLDR,0.234303


### Create 100 new dataframes for each stock: monthly data

In [71]:
# A 100-stocks list
ticker = stockret.TICKER.unique().tolist()

In [72]:
# Create 100 new stock dataframes and store into a dictionary
stockret_dict = {}
for i in ticker:
  stockret_dict[i] = stockret[stockret['TICKER'] == i]

In [88]:
# All 100 stock tickers
stockret_dict.keys()

dict_keys(['BWXT', 'BCPC', 'CAL', 'BC', 'BAH', 'BKU', 'BCOV', 'BLMN', 'BERY', 'BFAM', 'BCC', 'BLUE', 'BNFT', 'BRX', 'BURL', 'BRG', 'BLBD', 'BWFG', 'BCLI', 'BGSF', 'BOOT', 'CALA', 'BOX', 'BPMC', 'CABO', 'BLD', 'BATRA', 'BATRK', 'BL', 'BPOP', 'BOH', 'BCO', 'BRC', 'BMY', 'BA', 'CACI', 'CALM', 'CAMP', 'CAH', 'BAX', 'BOOM', 'BEN', 'BDX', 'B', 'BK', 'BMI', 'CAG', 'BRT', 'BLL', 'BAC', 'BKH', 'BXMT', 'BRO', 'BIG', 'BDN', 'C', 'BLFS', 'BHE', 'BIIB', 'BOKF', 'BKE', 'BSX', 'BBBY', 'CACC', 'CAKE', 'BLX', 'BANF', 'BBSI', 'BWA', 'BFS', 'BDC', 'BYD', 'BZH', 'BCRX', 'BANR', 'BJRI', 'BXP', 'BAM', 'BHB', 'CAC', 'BRKL', 'BBY', 'BMTC', 'BELFB', 'BUSE', 'BCOR', 'BSRR', 'BMRN', 'BLK', 'BMRC', 'BGCP', 'BHLB', 'BRKR', 'BG', 'BDSI', 'BANC', 'BLKB', 'BECN', 'BFIN', 'BLDR'])

In [96]:
# Display the first 180 rows (3 DataFrames:'BWXT', 'BCPC', 'CAL')
list(stockret_dict.values())[:3]

[    PERMNO      date TICKER       RET
 0    10220  20170131   BWXT  0.045088
 1    10220  20170228   BWXT  0.119306
 2    10220  20170331   BWXT  0.026916
 3    10220  20170428   BWXT  0.032983
 4    10220  20170531   BWXT -0.009355
 5    10220  20170630   BWXT  0.003086
 6    10220  20170731   BWXT  0.080615
 7    10220  20170831   BWXT  0.040812
 8    10220  20170929   BWXT  0.023757
 9    10220  20171031   BWXT  0.069618
 10   10220  20171130   BWXT  0.044059
 11   10220  20171229   BWXT -0.031385
 12   10220  20180131   BWXT  0.048768
 13   10220  20180228   BWXT -0.007566
 14   10220  20180329   BWXT  0.011595
 15   10220  20180430   BWXT  0.067212
 16   10220  20180531   BWXT -0.013274
 17   10220  20180629   BWXT -0.066227
 18   10220  20180731   BWXT  0.055199
 19   10220  20180831   BWXT -0.065085
 20   10220  20180928   BWXT  0.019896
 21   10220  20181031   BWXT -0.065238
 22   10220  20181130   BWXT -0.223743
 23   10220  20181231   BWXT -0.154578
 24   10220  20190131   B

###Upload Fama-French monthly risk factor data
* Fama French Risk Factors for 2017-2021 from the file "FF-Factors-2017-2021.csv"

In [73]:
ff_factors = pd.read_csv('/content/drive/MyDrive/BA_870/HW/3/FF-Factors-2017-2021.csv')

#### List varaibles in FF dataframe

In [74]:
ff_factors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   dateff  60 non-null     int64  
 1   mktrf   60 non-null     float64
 2   smb     60 non-null     float64
 3   hml     60 non-null     float64
 4   rf      60 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 2.5 KB


####Look at head and tail of dataframe

In [75]:
ff_factors.head()

,dateff,mktrf,smb,hml,rf
0,20170131,0.0194,-0.0113,-0.0274,0.0004
1,20170228,0.0357,-0.0204,-0.0167,0.0004
2,20170331,0.0017,0.0113,-0.0333,0.0003
3,20170428,0.0109,0.0072,-0.0213,0.0005
4,20170531,0.0106,-0.0252,-0.0375,0.0006


In [76]:
ff_factors.tail()

,dateff,mktrf,smb,hml,rf
55,20210831,0.0290,-0.0048,-0.0013,0.0000
56,20210930,-0.0437,0.0080,0.0509,0.0000
57,20211029,0.0665,-0.0228,-0.0044,0.0000
58,20211130,-0.0155,-0.0135,-0.0053,0.0000
59,20211231,0.0310,-0.0157,0.0323,0.0001


#### Rename date column to "date" to match WRDS data "date" column for 100 stocks

In [77]:
ff_factors.rename(columns={'dateff':'date'}, inplace=True)
ff_factors.head()

,date,mktrf,smb,hml,rf
0,20170131,0.0194,-0.0113,-0.0274,0.0004
1,20170228,0.0357,-0.0204,-0.0167,0.0004
2,20170331,0.0017,0.0113,-0.0333,0.0003
3,20170428,0.0109,0.0072,-0.0213,0.0005
4,20170531,0.0106,-0.0252,-0.0375,0.0006


### Merge the 100 stock return data and Fama-French market data based on "date"
* Then list head and tail of dataframe for BLDR

In [78]:
stockret_ff = {}
for i in stockret_dict.keys():
  stockret_ff[i] = pd.merge(stockret_dict[i], ff_factors, on = 'date', how = 'outer')

In [79]:
stockret_ff['BLDR'].head()

,PERMNO,date,TICKER,RET,mktrf,smb,hml,rf
0,90720,20170131,BLDR,-0.019143,0.0194,-0.0113,-0.0274,0.0004
1,90720,20170228,BLDR,0.202602,0.0357,-0.0204,-0.0167,0.0004
2,90720,20170331,BLDR,0.151468,0.0017,0.0113,-0.0333,0.0003
3,90720,20170428,BLDR,0.074497,0.0109,0.0072,-0.0213,0.0005
4,90720,20170531,BLDR,-0.146783,0.0106,-0.0252,-0.0375,0.0006


In [80]:
stockret_ff['BLDR'].tail()

,PERMNO,date,TICKER,RET,mktrf,smb,hml,rf
55,90720,20210831,BLDR,0.197528,0.0290,-0.0048,-0.0013,0.0000
56,90720,20210930,BLDR,-0.029086,-0.0437,0.0080,0.0509,0.0000
57,90720,20211029,BLDR,0.126208,0.0665,-0.0228,-0.0044,0.0000
58,90720,20211130,BLDR,0.191694,-0.0155,-0.0135,-0.0053,0.0000
59,90720,20211231,BLDR,0.234303,0.0310,-0.0157,0.0323,0.0001


##Run OLS regression for 100 stocks (60 months) using FF 3-factor model:
* [Ret(stock)-Rf] = alpha + B1(RetMkt-Rf) + b2(SMB) + b3(HML) + e

In [81]:
# Create a empty output dataframe
output = pd.DataFrame(columns = ['TICKER', 'R-squared', 'Adj. R-squared', 'const', 'mktrf', 'smb', 'hml'])
output

,TICKER,R-squared,Adj. R-squared,const,mktrf,smb,hml


In [82]:
# Define a Linear Regression function for FF model
def ffmodel(data,i):
  y = data[i]["RET"] - data[i]["rf"]
  X = data[i][['mktrf' , 'smb' , 'hml']] 
  # Use statsmodels
  X = sm.add_constant(X) # adding a constant
  model = sm.OLS(y, X).fit()

  #return regression output
  return (i, model.rsquared, model.rsquared_adj, model.params[0], model.params[1],model.params[2],model.params[3])

for i in stockret_ff.keys():
  output.loc[len(output.index)] = ffmodel(stockret_ff, i)

# Display the output regression statistics
output



/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,TICKER,R-squared,Adj. R-squared,const,mktrf,smb,hml
0,BWXT,0.319518,0.283063,-0.008452,1.035051,-0.255460,-0.025344
1,BCPC,0.151024,0.105543,0.007626,0.477501,0.231980,0.122664
2,CAL,0.491911,0.464692,-0.008616,2.010956,1.691033,1.102032
3,BC,0.614279,0.593615,-0.001431,1.446841,0.968595,0.372144
4,BAH,0.333477,0.297770,0.002267,0.764677,-0.241293,-0.486009
...,...,...,...,...,...,...,...
95,BANC,0.646651,0.627721,-0.001240,1.230841,1.623205,0.772896
96,BLKB,0.367581,0.333701,-0.007162,0.931270,0.530980,-0.047068
97,BECN,0.635610,0.616089,-0.007835,1.774667,0.088601,0.971480
98,BFIN,0.329431,0.293508,-0.003914,0.436431,0.576216,0.514933


In [83]:
# Store the output into a csv file
output_file = output.to_csv('Assign3-Output.csv', index = False)

In [ ]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

In [97]:
!jupyter nbconvert --to pdf '/content/drive/MyDrive/BA_870/HW/3/Assignment3_Ji_Qi.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/BA_870/HW/3/Assignment3_Ji_Qi.ipynb to pdf
[NbConvertApp] Writing 50495 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 67805 bytes to /content/drive/MyDrive/BA_870/HW/3/Assignment3_Ji_Qi.pdf
